In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [2]:
def wait1(s):
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def wait2(s):
    pass
wait = wait1

def call(f, *args):
    s = "(" + ", ".join(map(repr, args)) + ")"
    print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        else:
            return rv

### Global functions and routines

In [3]:
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

def settemp(sp):
    if sp == 0:
        call(h.settemp, 2, 37)
    else:
        call(h.settemp, 0, sp)
        
def endbatch():
    call(h.endbatch)
    
def pid1():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 12)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 29)
    
def pid2():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 13)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 26)
    
def pid3():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 14)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 26)

In [4]:
_bni = 1
def nb(pref, i=1, n=1):
    bl = call(h.getBatches)
    b = bl.names_to_batches
    s = "%s %d.%d" % (pref, i, n)
    while s in b:
        n += 1
        s = "%s %d.%d" % (pref, i, n)
    startbatch(s)

def tempoff():
    settemp(0)
    
def setbn(i):
    global _bni
    _bni = i

def o2a():
    nb("tpid", _bni)
    settemp(37)
    wait(3*hours)
    endbatch()
    
def cool(hrs=8):
    tempoff()
    wait(hrs*hours)
    
def a2a(end):
    nb("tpid", _bni)
    settemp(end)
    wait(3*hours)
    endbatch()

def verify(pid):
    print("Verifying settings")
    pid()
    o2a()
    a2a(35)
    a2a(37)
    
def pid4():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 13)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 28) 

## Round 4 5/24/17
Temp looked a bit unstable in real life in the bio lab. According to simulation, a much higher gain value doens't actually hurt performance, so trying the test out again. 

In [5]:
def pid5():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 50)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 26) 
hours = 3600

In [ ]:
h = HelloApp('71.189.82.196:81')
wait = wait1
cool(10)
setbn(1)
verify(pid5)
tempoff()

Calling settemp(2, 37)
Sleeping 36000 seconds, ending at 05/25/2017 03:16:34 AM
